# Gradient and AutoGrad

## AUTOGRAD: 자동 미분

- autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공한다. 
- 이는 define-by-run이라는 프레임 워크로 코드를 어떻게 작성하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라진다. 

## Tensor

- 패키지 중심에는 torch.Tensor 클래스가 있다.
- 만약 .requires_grad 속성을 True로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적학기 시작한다. 
- 계산이 완료된 후, .backward()를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있다.
- 이 Tensor의 변화도는 .grad 속성에 누적된다.
- Tensor가 기록을 추적하는 것을 중단하게 하려면, .detach()를 호출하여 연산 기록으로부터 분리하여 연산들이 추적되는 것을 방지할 수 있다. 
- 기록을 추적하는 것(메모리를 사용)을 방지하기 위해, 코드 블럭을 with torch.no_grad()로 감쌀 수 있다.
- 이는 특히 gradient는 필요없지만, require_grad = True 가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가할 때 유용하다. 

- Autograd 구현에서 중요한 클래스가 하나 더 있는데, 바로 Function 클래스이다. 
- Tensor와 Function은 상호 연결 되어있으며, 모든 연산 과정을 부호화하여 acycllic graph를 생성합니다. 
- 각 tensor는 .grad_fn 속성을 갖고 있는데, 이는 Tensor를 생성한 Function을 참조하고 있다. (사용자가 만든 텐서는 예외로 이 때 grad_fn은 None이다.)


- derivative를 계산하긱 위해서는 Tensor의 .backward()를 호출하면 된다. 
- 만약 Tensor가 스칼라인 경우에는 backward에 인자를 정해줄 필요가 없다. 하지만 여러 개 요소를 갖고 있을 때는 tensor의 모양을 gradient 인자로 지정할 필요가 있다. 


In [1]:
import torch

tensor 생성 후, requires_grad= True 를 설정하여 연산을 기록한다.


In [2]:
x= torch.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


tensor에 연산을 수행한다.

In [3]:
y = x + 2
print(y)


tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


- y는 연산의 결과로 생성된 것이므로 grad_fn 을 가진다.
- 반면 x는 사용자가 만든 tensor이므로 grad_fn은 None을 가진다.

In [4]:
print(y.grad_fn)
print(x.grad_fn)

None


In [5]:
z = y*y*3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


- .requires_grad_()는 기존 Tensor의 requires_grad 값을 in-place하여 변경한다..
- 입력값이 지정되지 않으면 기본값은 False이다. 


In [6]:
a = torch.randn(2,2)
print(a)
a = ((a*3) /(a-1))
print(a)
print(a.requires_grad) # false
print(a.grad_fn) # None 사용자가 직접만든 tensor이므로 
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a*a).sum() 
print(b.requires_grad) # False
print(b.grad_fn) # sumbackward

tensor([[ 0.1818,  0.6418],
        [-0.5078,  1.0462]])
tensor([[-0.6664, -5.3742],
        [ 1.0104, 67.9117]])
False
None
True
True


## 변화도 (GRADIENT)


- 이제 역전파(backprop)를 해보겠다. out은 하나의 스칼라 값만 갖고 있기 때문에
- out.backward()는 out.backward(torch.tensor(1.))과 동일하다. 

In [7]:
out.backward()

변화도 d(out)/dx를 출력한다


In [8]:
print(x.grad) # partial o / partial x_i

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


- 수학적으로 벡터 함수 𝑦⃗ =𝑓(𝑥⃗ ) 에서 𝑥⃗  에 대한 𝑦⃗  의 변화도는 야코비안 행렬(Jacobian Matrix)입니다:
- 일반적으로, torch.autograd는 벡터-야코비안 곱을 계산하는 엔진이다. 

In [9]:
x =  torch.randn(3, requires_grad =True)
y = x *2
while y.data.norm() < 1000:
    y= y*2

print(x)
print(y)

tensor([ 0.0617, -0.5153,  0.5985], requires_grad=True)
tensor([  126.4052, -1055.3285,  1225.8296], grad_fn=<MulBackward0>)


- 이 경우 y는 더 이상 스칼라 값이 아니다. torch.autograd는 전체 야코비안을 직접 계산할 수 없지만, 벡터 야코비안 곱은 간단히 backward에 해당 벡터를 인자로 제공하여 얻을 수 있다.

In [10]:
v = torch.tensor([0.1,1.0, 0.0001], dtype = torch.float)
y.backward(v)
print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


- 또한 with torch.no_grad(): 로 코드 블럭을 감싸서 
- autograd가 .requires_grad=True 인 텐서의 연산기록을 추적하는 것을 멈출 수 있다. 

In [11]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False
